In [2]:
from osgeo import gdal
import numpy
from numpy.core.umath import add, subtract


class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def Band2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, rows)
        array= array.reshape(cols*rows)
        return  array
    
    def trtData(self,bs):
        self.bs=bs
        X=[]
        for i in range(len(bs)):
            A=[]
            A.append(bs[i])
            X.append(A)
        return X

    def trainResult(self,flood):
        self.flood=flood
        Y=[]
        for i in range(len(flood)):
            Y.append(flood[i])
        return Y

    def index2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, rows)
        array = array.reshape(cols * rows)
        return  array
    
    def index2Array2(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        self.fill=fill
        array = band.ReadAsArray(0, 0, cols, (rows-1))
        array = array.reshape(cols * (rows-1))
        return  array

    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        self.outArray = outArray
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Float32)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand

    def cMatrix(self, actual, test):
        self.actual=actual
        self.test=test
        F_F=0
        F_N=0
        F_fi=0
        N_F =0
        N_N =0
        N_fi=0
        Fi_F=0
        Fi_N=0
        Fi_fi=0
    
        for i in range(len(actual)):
            if actual[i]==test[i]:
                if actual[i]==0:
                    N_N += 1
                elif actual[i]==1:
                    F_F += 1
                else:
                    Fi_fi += 1
            else:
                if actual[i]==0:
                    if test[i]==1:
                        N_F += 1
                    else:
                        N_fi+= 1
                elif actual[i]==1:
                    if test[i]==0:
                        F_N +=1
                    else:
                        F_fi +=1 
                else:
                    if test[i]==1:
                        Fi_F +=1
                    else:
                        Fi_N +=1
        
        LIST= [F_F,F_N,F_fi,N_F,N_N,
               N_fi,Fi_F,Fi_N,Fi_fi]
        
        C_matrix = numpy.array(LIST).reshape(3,3)
        return C_matrix
    
    def testData(self, evi, lswi, dvel):
        self.evi = evi
        self.lswi = lswi
        self.dvel = dvel
        limage = self.Index(evi)
        cols = self.iminfo['cols']
        # Get pixel row number
        rows = self.iminfo['rows']
        bandnum= self.iminfo['bandnum']
        EVI = (limage.index2Array(fill))
        limage = Index(lswi)
        LSWI = (limage.index2Array(fill))
        limage = Index(dvel)
        DVEL = (limage.index2Array(fill))
        data = (limage.trtData(EVI, LSWI, DVEL))


In [ ]:

import numpy as np
from sklearn import svm
from osgeo import gdal
from sklearn.externals import joblib
from multiprocessing import Pool

fill=-999
path="/home/faizan/Desktop/full/"
np_dir="/home/faizan/Desktop/numpy_dir/"
"""
#assigning names training data

bs_t=path+"BS_t_hv.tif"
flood_t=path+"PFlood.tif"

#loading image training
limage = Index(bs_t)
BS_t = (limage.index2Array(fill))

limage = Index(flood_t)
Flood_t = (limage.index2Array(fill))

# arranging training data
X = (limage.trtData(BS_t))
Y = (limage.trainResult(Flood_t))
# SVM machine learning

clf = svm.SVC()
clf = clf.fit(X,Y)

#save model

joblib.dump(clf, '/home/faizan/Desktop/full/Palsar_hv_finaltraining.pkl')
del BS_t, Flood_t, X, Y

"""

#assigning names test data
bs=path+"HH.ALPSRP245160540.backscatter_try.tif"
Oflood = path+"HH.ALPSRP245160540_flood.tif"

#bs=path+"238540.tif"
#Oflood = path+"238540_flood1.tif"
#save model

#joblib.dump(clf, '/home/faizan/Desktop/full/trained.pkl')
clf = joblib.load('/home/faizan/Desktop/full/Palsar_finaltraining.pkl') 

#del EVI_t, LSWI_t, DVEL_t, Flood_t, X, Y
#Test data
limage = Index(bs)

cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
BS = (limage.index2Array(fill))

Data = (limage.trtData(BS))

del BS

def argwrapper(args):
    '''
    ラッパー関数
    '''
    return args[0](*args[1:])

def myfunc(x):
    '''
    並列に計算したい関数
    ''' 
    return clf.predict(x)
import time
start_time = time.time()



if __name__ == '__main__':
    p = Pool(6)
    for a in xrange(0,50):
        func_args = []
        for i in xrange((len(Data)*a)/50, (len(Data)*(a+1))/50):
            X= Data[i]
            func_args.append( (myfunc, X) )
        results = numpy.array(p.map(argwrapper, func_args), dtype=numpy.int16)
        del func_args
        numpy.save(np_dir + "svr_result%02d" % a, results)
        del results 
        print("--- %s seconds ---" % (time.time() - start_time))
    del Data
    # Load all array and merge
    # load first array
    ARRAY = numpy.load(np_dir + "svr_result00.npy")

    for a in range(1,50):
        dummy = numpy.load(np_dir + "svr_result%02d.npy" % (a))
        ARRAY = numpy.vstack([ARRAY, dummy])
    limage.WriteArrayAsImage(Oflood, ARRAY)
    del ARRAY, dummy
"""
if __name__ == '__main__':
    p = Pool(6)
    for a in xrange(0,20):
        func_args = []
        for i in range(6):
            S = (len(Data)*(a))/20
            I = (((len(Data)*(a+1))/20)- ((len(Data)*a)/20))/6
            X = Data[(S+(I*i)): (S+(I*(i+1)))]
            #X = Data[(len(Data)*a)/20 : (len(Data)*(a+1))/20]
            func_args.append( (myfunc, X) )
        results =(p.map(argwrapper, func_args))
        del func_args
        numpy.save(np_dir + "svr_result%02d" % a, results)
        del results 
        print("--- %s seconds ---" % (time.time() - start_time))
    del Data
    # Load all array and merge
    # load first array
    ARRAY = numpy.load(np_dir + "svr_result00.npy")

    for a in range(1,20):
        dummy = numpy.load(np_dir + "svr_result%02d.npy" % (a))
        ARRAY = numpy.vstack([ARRAY, dummy])
    limage.WriteArrayAsImage(Oflood, ARRAY)
    del ARRAY, dummy
"""
print("--- %s seconds ---" % (time.time() - start_time))





In [20]:
from sklearn.externals import joblib
from sklearn import svm
clf = joblib.load('/home/faizan/Desktop/full/Palsar_hv_finaltraining.pkl') 


In [49]:
clf.predict(-29.4793)

array([ 1.], dtype=float32)

In [7]:
A

(0, 1760642)

In [6]:
# Load all array and merge
# load first array
ARRAY = numpy.load(np_dir + "svr_result00.npy")

for a in range(1,20):
    dummy = numpy.load(np_dir + "svr_result%02d.npy" % (a))
    ARRAY = numpy.vstack([ARRAY, dummy])
limage.WriteArrayAsImage(Oflood, ARRAY)
#del ARRAY

In [24]:
def argwrapper(args):
    '''
    ラッパー関数
    '''
    return args[0](*args[1:])

def trfunc(X,Y):
    
 
    #clf = svm.SVC()
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X,Y)
    return clf

if __name__ == '__main__':
    from sklearn import tree
    p = Pool(6)
    func_args = []
    for a in xrange(0,1):
        for i in xrange((len(X)*a)/1200, (len(X)*(a+1))/1200):
            x= X[i]
            y= [Y[i]]
            func_args.append( (trfunc, x, y) )
    clf = p.map(argwrapper, func_args)
    #del func_args, x, y
    #joblib.dump(clf, '/home/faizan/Desktop/full/Palsar_svm_training.pkl')
    #del BS_t, Flood_t, X, Y


In [37]:
>>> from sklearn import svm
>>> X = [[0, 0], [1, 1]]
>>> y = [0, 1]
>>> clf = svm.SVC()
>>> clf.fit(X, y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [37]:
cMat= limage.cMatrix(Actual, test)
cMat

array([[  70749,    2309,       0],
       [   2248,  502907, 1195120],
       [      0,       0,   66535]])